In [1]:

import pandas as pd
import numpy as np

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import os

import util

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Available device: {}".format(device))

# training hyperparameters
learning_rate = 1e-1
epochs = 100
decay_step = [50, 75]
decay_rate = 0.1
opt = 'Adam'

# data loading
batch_size = 1
num_workers = 4
csv_dir = './archive'

train_ds = util.MITBIH_DS(csv_dir, 'mitbih_train.csv')
valid_ds = util.MITBIH_DS(csv_dir, 'mitbih_test.csv')

train_loader = DataLoader(train_ds, shuffle=False)



Available device: cpu


In [3]:

"""
for i in range(0, len(train_ds), 5000):
    ecg, lbl = train_ds[i]
    util.plot_ecg(ecg, 'Label: {}'.format('N S V F Q'.split()[lbl]))
"""
;

''

In [4]:

%%capture

class TNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.training = True
        # out = (in - kern) / stride + 1
        self.conv = nn.Sequential(  # input: 1, 186
            nn.Conv1d(1, 32, kernel_size=9, stride=3),  # 32, 60
         	nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=6, stride=2),    # 64, 28
         	nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=4, stride=1),    # 128, 25
         	nn.ReLU(),
            nn.Conv1d(128, 256, kernel_size=3, stride=2),    # 256, 12
         	nn.ReLU()
        )
        self.fc = nn.Sequential(
            nn.Linear(3072, 64),
            nn.ReLU(),
            nn.Linear(64, 5),
            nn.ReLU(),
            nn.Softmax(dim=-1),
        )

    def forward(self, x):
        x = self.conv(x)
        x = torch.reshape(x, (1, 1, 3072))
        x = self.fc(x)
        return x

class ConvNet(nn.Module):

    def __init__(self, num_of_class: int) -> None:

        super(ConvNet, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),
        )

        self.fc = nn.Sequential(
            nn.Linear(2944, 500),
            nn.LeakyReLU(inplace=True),
            nn.Linear(500, num_of_class),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.unsqueeze(1)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


net = ConvNet(5)
net.float()
net.train()
net.to(device)


In [5]:

t = torch.randn(1, 1, 186)
t.float()
out = net(t)
print('shape: {} out: {}'.format(out.shape, out))


RuntimeError: Expected 3-dimensional input for 3-dimensional weight [16, 1, 3], but got 4-dimensional input of size [1, 1, 1, 186] instead

In [9]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate,
                       amsgrad=True) if opt == 'Adam' else None

scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=decay_step, gamma=decay_rate)

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        #print("input type: {}, shape: {}".format(type(inputs), inputs.shape))
        #print("labels type: {}, shape: {}".format(type(labels), labels.shape))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #outputs = outputs.squeeze(dim=0)
        #labels = labels.squeeze(dim=0)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 500 == 499:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 500 : .3f}')
            running_loss = 0.0

print('Finished Training')



[1,   500] loss:  0.000


KeyboardInterrupt: 